In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

Populating the interactive namespace from numpy and matplotlib


## Create problem instance

In [2]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = CostObj()
# lrsegs = ['N42071SL2_2410_2700']
lrsegs = ['N51133RL0_6450_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)

# ---- Set the tau target load ----
for k in data.tau:
    data.tau[k] = 12
    taustr = str(round(data.tau[k], 1))

# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)
# Print the target load reduction values
for l in mdl.LRSEGS:
    for p in mdl.PLTNTS:
        print('%s: %d' % (mdl.tau[l,p], mdl.tau[l,p].value))

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

tau[N51133RL0_6450_0000,P]: 12
tau[N51133RL0_6450_0000,S]: 12
tau[N51133RL0_6450_0000,N]: 12


In [3]:
tol = 1e-2
nzvarnames = []
nzvarvalus = []
i=0
for k in mdl.x.keys():
#     print(k)
    if not not mdl.x[k].value:
#         print(mdl.x[k].value)
        if abs(mdl.x[k].value) > tol:
            nzvarnames.append(mdl.x[k].getname())
            nzvarvalus.append(mdl.x[k].value)
    i+=1
#     if i > 100:
#         break
print(i)

7626


In [4]:
# for index in mdl.x:
#     mdl.x[index].value=0
#             x_value = oe.value(v[index])

# reinitialize the variables
for k in mdl.x:
#         mdl.x[k] = float(random.randrange(0, 600001))/100
    mdl.x[k] = round(random.uniform(0, 6000), 2)

In [5]:
# mdl.x.pprint()

## Solve problem instance

In [6]:
looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')

myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)
# set filepath for saving information about all of the solver iterates
output_file_name=os.path.join(projectpath, ''.join(['output/single_CostObj_', looptimestamp, '.iters']))
myobj.modify_ipopt_options(optionsfilepath='ipopt.opt', newoutputfilepath=output_file_name)

merged_df = myobj.solve()
print('\nObjective is: %d' % oe.value(mdl.Total_Cost))

Solver log file: 'logfile_loadobjective.log'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmphe90cxd1.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmphe90cxd1.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     3609
Number of nonzeros in Lagrangian Hessian.............:     5575

Total number of variable

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  7.1219411e+06 1.95e+03 4.19e+03  -1.0 1.90e+04    -  7.80e-04 5.30e-02f  1
  81  7.1199392e+06 1.95e+03 4.26e+03  -1.0 3.56e+04    -  9.17e-03 1.63e-04f  1
  82  7.1173667e+06 1.95e+03 3.05e+03  -1.0 1.82e+04    -  3.82e-02 1.09e-03f  1
  83  6.9130913e+06 1.77e+03 2.79e+03  -1.0 1.81e+04    -  3.30e-02 9.19e-02f  1
  84  6.9110838e+06 1.77e+03 2.79e+03  -1.0 1.65e+04    -  1.44e-01 1.01e-03f  1
  85  6.9077408e+06 1.77e+03 2.78e+03  -1.0 1.69e+04    -  1.33e-02 1.52e-03f  1
  86  5.7779665e+06 8.85e+02 1.39e+03  -1.0 1.78e+04    -  4.62e-03 4.99e-01f  1
  87  5.2795136e+06 4.60e+02 7.25e+02  -1.0 8.89e+03    -  1.40e-01 4.80e-01f  1
  88  5.1284058e+06 3.51e+02 5.71e+02  -1.0 5.16e+03    -  1.63e-03 2.36e-01f  1
  89  5.1258994e+06 3.50e+02 5.51e+02  -1.0 4.48e+03  -4.0 4.82e-01 4.07e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  4.8795576e+06 3.49e+02

 172  2.8277029e+06 0.00e+00 2.10e+03  -8.6 2.53e+00    -  9.66e-01 9.96e-01f  1
 173  2.8277029e+06 0.00e+00 4.17e+01  -8.6 5.67e-01    -  9.80e-01 9.99e-01f  1
 174  2.8277029e+06 0.00e+00 2.99e+01  -8.6 2.46e-05    -  2.82e-01 1.00e+00f  1
 175  2.8277029e+06 0.00e+00 2.37e+01  -8.6 1.83e-07    -  2.09e-01 1.00e+00f  1
 176  2.8277029e+06 0.00e+00 7.74e+00  -8.6 3.86e-08    -  6.73e-01 1.00e+00f  1
 177  2.8277029e+06 0.00e+00 3.20e+00  -8.6 1.09e-07    -  5.87e-01 1.00e+00f  1
 178  2.8277029e+06 0.00e+00 5.20e-01  -8.6 8.24e-08    -  8.37e-01 1.00e+00h  1
 179  2.8277029e+06 0.00e+00 4.26e-14  -8.6 3.76e-09    -  1.00e+00 1.00e+00f  1

Number of Iterations....: 179

                                   (scaled)                 (unscaled)
Objective...............:   1.9893759250380648e+04    2.8277029186009555e+06
Dual infeasibility......:   4.2632564145606011e-14    6.0598011941692672e-12
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity...

In [7]:
asdkjhg
from util.solution_wrangler import get_nonzero_var_names_and_values
nzvnames, nzvvalues = get_nonzero_var_names_and_values(mdl)
print(len(nzvnames))
nzvnames[1]

NameError: name 'asdkjhg' is not defined

In [ ]:
for ii in mdl.x.iteritems():
    print(ii)

In [ ]:
tol = 1e-2
nzvarnames = []
nzvarvalus = []
i=0
for k in mdl.x.keys():
#     print(k)
    if not not mdl.x[k].value:
#         print(mdl.x[k].value)
        if abs(mdl.x[k].value) > tol:
            nzvarnames.append(mdl.x[k].getname())
            nzvarvalus.append(mdl.x[k].value)
    i+=1
#     if i > 100:
#         break
print(i)

In [ ]:
print(output_file_name)
dict_of_iterates = myobj.parse_output_file(output_file_name)
# dict_of_iterates.keys()

In [ ]:
varvals = {}
varvals[0] = []
for ii in dict_of_iterates.keys():
    df = dict_of_iterates[ii]
#     display(df.head(5))
    val = float(df.loc[(df.outputname=='curr_x') &
                       (df.varname=='x') & 
                       (df.index.get_level_values('varindex')=='[ConPlan,N42071SL2_2410_2700,pas]')]['value'][0])
#     print(val)
    varvals[0].append(val)

varvals[1] = []
for ii in dict_of_iterates.keys():
    df = dict_of_iterates[ii]
    varvals[1].append(float(df.loc[(df.outputname=='curr_x') &
                                   (df.varname=='x') & 
                                   (df.index.get_level_values('varindex')=='[UrbanNMPlanHR,N42071SL2_2410_2700,ntg]')]['value'][0]))

In [ ]:
# Make Figure

fig = plt.figure(figsize=(10, 4))
rects = plt.plot(range(len(varvals[0])), varvals[0])
ax = plt.gca()
plt.ylabel('varvals[0]')
plt.xlabel('iteration')

fig = plt.figure(figsize=(10, 4))
rects = plt.plot(range(len(varvals[1])), varvals[1])
ax = plt.gca()
plt.ylabel('varvals[1]')
plt.xlabel('iteration')

fig = plt.figure(figsize=(10, 4))
rects = plt.scatter(varvals[0], varvals[1], c=range(len(varvals[1])))
ax = plt.gca()
plt.colorbar()
plt.xlabel('varvals[0]')
plt.ylabel('varvals[1]')

## Visualize

In [ ]:
merged_df.tail(50)

In [ ]:
for l in mdl.LRSEGS:
    for p in mdl.PLTNTS:
        print('%s: %d' % (mdl.TargetPercentReduction[l,p], oe.value(mdl.TargetPercentReduction[l,p].body)))

In [ ]:
from datetime import datetime
# ---- Make zL Figure ----
filenamestr = ''.join(['output/costobj_zL_tau', taustr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'),
                       '.png'])
savefilepathandname = os.path.join(projectpath, filenamestr)

zL_bars(df=merged_df, instance=mdl,
        savefig=True, savefilepathandname=savefilepathandname)

In [ ]:
# ---- Acres Figure ----
sorteddf_byacres = merged_df.sort_values(by='acres')

filenamestr = ''.join(['output/costobj_x_tau', taustr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.png'])
savefilepathandname = os.path.join(projectpath, filenamestr)

objstr = ''.join(['Objective is: ', str(round(mdl.Total_Cost(), 2))])
titlestr = '\n'.join([objstr, 'labels are (cost per unit, total bmp instance cost)'])

acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
           savefig=True, savefilepathandname=savefilepathandname)

## Gradient, Jacobian, Hessian

In [ ]:
gjh_filename, g = gjh_solve(instance=mdl,
                            keepfiles=True,
                            amplenv=ampl,
                            basegjhpath=os.getcwd())

g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

g_df = sorteddf_byacres.merge(g_df, how='left',
                              on=['bmpshortname', 'landriversegment', 'loadsource'],
                              sort=False)

In [ ]:
# ---- Make gradient Figure ----
# g_df_filtered = g_df.loc[abs(g_df['g'])>0.001,:].copy()
g_df_filtered = g_df
# sorteddf = g_df_filtered.sort_values(by='g')
# sorteddf_byacres

# sorteddf_byacres
# g_df = g_df.merge(sorteddf_byacres, how='right',
#                on=['bmpshortname', 'landriversegment', 'loadsource'])

keystrs = [str([x, y]) for x, y in zip(g_df_filtered['bmpshortname'], g_df_filtered['loadsource'])]
# Make Figure
fig = plt.figure(figsize=(10, 4))
rects = plt.barh(y=keystrs, width=g_df_filtered['g'])
ax = plt.gca()

ax.set_position([0.3,0.1,0.5,0.8])


filenamestr = ''.join(['output/costobj_g_tau', taustr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'),
                       '.png'])
plt.savefig(os.path.join(projectpath, filenamestr))

In [ ]:
# #Read number of variables and constraints
# nl_file = open(''.join([filecode, '.pyomo.nl']),'r')
# nl_lines = nl_file.readlines()
# num_variables = int(nl_lines[1].split(' ')[1])
# num_constraints = int(nl_lines[1].split(' ')[2])

In [ ]:
for v in mdl.component_objects(oe.Var, active=True):
    print ("Variable component object",v)
    i=0
    for index in v:
        try:
            x_value = oe.value(v[index])
            i+=1
            try:
                print (i,"   ", index, v[index].value, g[i])
            except:
                print (i,"   ", index, v[index].value)
        except:
            pass

In [ ]:
# #Read number of variables and constraints
# gjh_file = open(''.join([filecode, '.pyomo.gjh']),'r')
# gjh_lines = gjh_file.readlines()

In [ ]:
# print(gjh_lines[1])

In [ ]:
# num_variables = int(gjh_lines[1].split(' ')[1])
# num_constraints = int(gjh_lines[1].split(' ')[2])

In [ ]:
# #Read gjh output file
# gjh = oe.AbstractModel()
# # gjh.n_var = oe.Set(initialize=range(1,num_variables+1))
# # gjh.n_cons = oe.Set(initialize=range(1,num_constraints+1))
# # gjh.g = oe.Param(gjh.n_var, default=0)
# # gjh.J = oe.Param(gjh.n_cons, gjh.n_var, default=0)
# # gjh.H = oe.Param(gjh.n_var, gjh.n_var, default=0)
# # os.rename(''.join([filecode, '.pyomo.gjh']), ''.join([filecode, '.pyomo.gjh.dat']))
# gjh_ins = gjh.create_instance(''.join([filecode, '.pyomo.gjh.dat']))

# #Print
# for i in gjh_ins.n_var:
#     print(gjh_ins.g[i])